In [121]:
import pandas as pd
import json
from functions.mongo import connectCollection
from bson.objectid import ObjectId

In [142]:
# connecting to database and charging the original database
db, coll = connectCollection('chats','messages')

with open('./input/chats.json') as f:
    chats_json = json.load(f)
coll.insert_many(chats_json)

In [135]:
data = list(coll.find({}))

In [136]:
namelst=[]
for i in range(len(data)):
    name = data[i]['userName']
    userid = data[i]['idUser']
    tup = (userid,name)
    namelst.append(tup)
namelst = set(namelst)
lst = []
for e in namelst:
    uid = e[0]
    name= e[1]
    dictionary = {
        'User_id':uid,
        'name': name
    }
    lst.append(dictionary)
lst

[{'User_id': 4, 'name': 'Ellen Ripley'},
 {'User_id': 0, 'name': 'John Wick'},
 {'User_id': 7, 'name': 'Danny Torrance'},
 {'User_id': 2, 'name': 'Leia Organa'},
 {'User_id': 3, 'name': 'Tony Stark'},
 {'User_id': 1, 'name': 'Mike Wazowski'},
 {'User_id': 6, 'name': 'Danny Torrances Finger'},
 {'User_id': 5, 'name': 'J. Jonah Jameson'}]

In [137]:
db, collus = connectCollection('chats','users')
with open('output/users.json', 'w') as fp:
    json.dump(lst, fp)

In [138]:
with open('./output/users.json') as f:
    users_json = json.load(f)
collus.insert_many(users_json)

In [139]:
usersdata = list(collus.find({}))
dic_ids = {}
for i in range(len(usersdata)):
    user = usersdata[i].get('User_id')
    objid = usersdata[i].get('_id')
    dic_ids[user]=objid
dic_ids
    

{4: ObjectId('5de167288a94224a1624bb11'),
 0: ObjectId('5de167288a94224a1624bb12'),
 7: ObjectId('5de167288a94224a1624bb13'),
 2: ObjectId('5de167288a94224a1624bb14'),
 3: ObjectId('5de167288a94224a1624bb15'),
 1: ObjectId('5de167288a94224a1624bb16'),
 6: ObjectId('5de167288a94224a1624bb17'),
 5: ObjectId('5de167288a94224a1624bb18')}

In [140]:
for i in range(len(data)):
    idu = dic_ids.get(data[i]['idUser'])
    value = {"$set":{'idUser':idu}}
    coll.update_one(data[i],value)